### Implementing gridworld example in the book:
<img src="./gridworld.png" alt="" width="60%"/>  

<div>At each cell, four actions are possible: north, south, east and west, which deterministically cause the agent to move one cell in respective direction on the grid. Actions that would take the agent off the grid leave its location unchanged, but also result in a reward of -1. Other actions result in a reward of 0, except those that move the agent out of the special states A and B. From state A, all four actions yield a reward of +10 and take the agent to A'. From state B, all actoins yield a reward of +5 and take the agent to B'.</div>

In [1]:
from bokeh.io import output_notebook
from bokeh.charts import Line, show

output_notebook()

Loading BokehJS ...

In [14]:
GAMMA = 0.9

In [60]:
def transition(s, a):
    """given state s and action a, return new state s' and reward r"""
    if s == (0,1): # A to A'
        return ((4,1), 10)
    elif s == (0,3):  # B to B'
        return ((2,3), 5)
    if a == "N":
        s_new = (s[0]-1, s[1])
    elif a == "S":
        s_new = (s[0]+1, s[1])
    elif a == "W":
        s_new = (s[0], s[1]-1)        
    elif a == "E":
        s_new = (s[0], s[1]+1)      
    else:
        raise Exception
    if (s_new[0] < 0 or s_new[0] > 4 or s_new[1] < 0 or s_new[1] > 4):
        return (s, -1)
    else:
        return (s_new, 0)
    
    
def update_value(s, grid):
    new_value = 0
    for a in ["N", "S", "E", "W"]:
        s_new, reward = transition(s, a)
        new_value += 0.25 * (reward + GAMMA * grid[s_new])
    return new_value


def value_iteration(iters, init):
    values = [init]
    for i in range(iters):
        new_grid = values[-1].copy()
        for row in range(5):
            for col in range(5):
                new_grid[row][col] = update_value((row, col), values[-1])
        values.append(new_grid)
    return values


def plot_value_change(data, values):
    p = Line(data, plot_width=400, plot_height=300, legend="bottom_right", 
         ylabel='grid value', xlabel="iterations")
    show(p)

In [61]:
# print grid values after 100 iterations and plot grid value change
initial = np.zeros((5,5))
values = value_iteration(100, initial)

for i in [0,1,2,5,10,50]:
    print("after {0} iteration, the grid values are:".format(i))
    print(value_trend[i],"\n")
        
data = {"A": [i[(0,1)] for i in values], 
        "B": [i[(0,3)] for i in values],
        "(0,2)": [i[(0,2)] for i in values]}
plot_value_change(data, values)

after 0 iteration, the grid values are:
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]] 

after 1 iteration, the grid values are:
[[ 3.30899932  8.78929484  4.42762216  5.32237057  1.49218174]
 [ 1.52159105  2.99232084  2.25014293  1.90757468  0.54740569]
 [ 0.05082547  0.73817357  0.67311624  0.35818919 -0.40313816]
 [-0.97358932 -0.43549245 -0.35487929 -0.58560211 -1.1830721 ]
 [-1.85769757 -1.34522828 -1.22926428 -1.42291517 -1.97517607]] 

after 2 iteration, the grid values are:
[[ 3.30899932  8.78929484  4.42762216  5.32237057  1.49218174]
 [ 1.52159105  2.99232084  2.25014293  1.90757468  0.54740569]
 [ 0.05082547  0.73817357  0.67311624  0.35818919 -0.40313816]
 [-0.97358932 -0.43549245 -0.35487929 -0.58560211 -1.1830721 ]
 [-1.85769757 -1.34522828 -1.22926428 -1.42291517 -1.97517607]] 

after 5 iteration, the grid values are:
[[ 3.30899932  8.78929484  4.42762216  5.32237057  1.49218174]
 [ 1.52159105  2.992320

### Finding the optimal solution in gridworld by solving bellman equation
<img src="./gridworld2.png" alt="" width="60%" align="left"/>  
<img src="./optimal_v.JPG" alt="" width="60%" align="left"/>  






In [126]:
""" define new value update function:
    instead of having equal chance to pick a action (N, W S or E), greedily 
    pick the action that would lead to highest reward
    if there are multiple best rewards, break tie by chosing first optimal value """

def optimal_value_update(s, grid):
    best_action = ""
    best_value = -100
    for a in ["N", "S", "E", "W"]:
        s_new, reward = transition(s, a)
        new_value = (reward + GAMMA * grid[s_new])
        if new_value > best_value:
            best_value = new_value
            best_action = a
    return best_action, best_value


def optimal_value_iteration(iters, init):
    values = [init]
    for i in range(iters):
        new_grid = values[-1].copy()
        for row in range(5):
            for col in range(5):
                new_grid[(row, col)] = optimal_value_update((row, col), values[-1])[1]
        values.append(new_grid)
    return values

In [127]:
# print optimal grid values after 100 iterations and plot grid value change
initial = np.zeros((5,5))
values = optimal_value_iteration(100, initial)

for i in [0,1,2,5,10,50]:
    print("after {0} iteration, the grid values are:".format(i))
    print(values[i],"\n")
        
data = {"A": [i[(0,1)] for i in values], 
        "B": [i[(0,3)] for i in values],
        "(0,2)": [i[(0,2)] for i in values],
        "(4,0)": [i[(4,0)] for i in values]}
p = Line(data, plot_width=400, plot_height=300, legend="bottom_right", 
         ylabel='grid value', xlabel="iterations")
show(p)

after 0 iteration, the grid values are:
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]] 

after 1 iteration, the grid values are:
[[  0.  10.   0.   5.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]] 

after 2 iteration, the grid values are:
[[  9.   10.    9.    5.    4.5]
 [  0.    9.    0.    4.5   0. ]
 [  0.    0.    0.    0.    0. ]
 [  0.    0.    0.    0.    0. ]
 [  0.    0.    0.    0.    0. ]] 

after 5 iteration, the grid values are:
[[  9.      10.       9.       8.645    7.7805]
 [  8.1      9.       8.1      7.7805   6.561 ]
 [  7.29     8.1      7.29     6.561    3.645 ]
 [  6.561    7.29     6.561    3.645    3.2805]
 [  0.       6.561    0.       3.2805   0.    ]] 

after 10 iteration, the grid values are:
[[ 14.31441     15.9049      14.31441     13.23930744  11.65470489]
 [ 12.882969    14.31441     12.882969    11.65470489  10.4